In [57]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [58]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [59]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = np.subtract(data,rf)

actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.74,-0.49,-1.68,2.02,-0.96,-0.50,-1.30,0.29,0.53,-1.88,-0.39
196308,4.62,6.30,5.95,3.68,4.89,4.04,6.15,9.31,3.95,5.24,5.07
196309,-1.95,-0.51,-1.03,-3.91,-0.14,2.09,0.69,-4.33,-2.77,-3.43,-1.57
196310,2.37,9.43,2.29,-0.61,8.00,3.11,0.19,3.09,-0.96,1.09,2.53
196311,-1.40,-5.11,0.03,-1.42,-0.56,3.89,-1.50,-1.92,-1.29,-0.04,-0.85


In [60]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [61]:
new_returns_data = returns_data
returns_monthly = new_returns_data
returns_mean = returns_monthly.mean()
cov_matrix = returns_monthly.cov()
    
# udregner profit, volatilitet og sharpe
tl = np.dot(np.linalg.inv(cov_matrix), returns_mean)
nl = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), returns_mean))
weights = tl / nl
    
# udregner afkast for måned T + 1 med de udregnede optimale vægte
returns = np.dot(weights, returns_mean)
variance = np.dot(weights, np.dot(cov_matrix, weights))
sharpe = returns / np.sqrt(variance)

# indsætter overstående udregninger i vektorerne
sharpe_ratio.append(sharpe)
port_returns.append(returns)
port_volatility.append(variance)
stock_weights.append(weights)

In [62]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Variance': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [63]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [64]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Variance', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Variance,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,1.172912,30.963999,0.210784,1.4686,-0.013496,0.368831,1.168235,1.194824,0.670754,0.419163,0.585276,0.061795,0.703926,-5.627908


In [65]:
df['Returns'].mean() / df['Returns'].std()

nan